# Notes / Code Following Along With "Building Quantum Software with Python"

Book available [here](https://www.manning.com/books/building-quantum-software-with-python?utm_source=stefanski&utm_medium=affiliate&utm_campaign=book_gonciulea_building_4_11_24&a_aid=stefanski&a_bid=2f351331&chan=mm_email&)

## What directions correspond to amplitudes that are real numbers?

In [ ]:
## The only directions that give purely real amplitudes are those that lie
## exactly on the real axis of the complex plane—i.e., phase angles θ = 0 rad
 ## (pointing right) or θ = π rad (pointing left).

## Write code that verifies that multiplying the 1 side of the following example state by –1 reverses its direction:

In [14]:
import math
state = [math.sqrt(0.3), -math.sqrt(0.7)]

In [17]:
direction = math.atan2(state[1].imag, state[1].real)

direction * (180/math.pi) # original direction

180.0

In [18]:
state = [math.sqrt(0.3), -math.sqrt(0.7)]

state[1] = state[1]* -1

direction = math.atan2(state[1].imag, state[1].real)

direction * (180/math.pi) # reversed direction

0.0

## Get the new value of the 1 side of a single-qubit amplitude pair after applying a phase gate:

In [32]:
# shortcut func for rotations
def cis(theta):
  return cos(theta) + 1j*sin(theta)

phi = pi/3
state = [state[0], cis(phi)*state[1]]

state

[(-0.34149942885245477+0.42822673911544473j),
 (-0.7643269668313427+0.34030029058818j)]

## HADAMARD GATE - replaces an amplitude pair with their sum and difference divided by the square root of 2,

In [33]:
state = [sqrt(0.5)*(state[0] + state[1]), sqrt(0.5)*(state[0] -state[1])]

state

[(-0.7819373432030916+0.5434306742285884j),
 (0.29898421937728414+0.062173387999278816j)]

## THE RZ GATE - rotates the 0 side of a pair of amplitudes clockwise by θ/2 and the 1 side of the pair counterclockwise by θ/2.

In [34]:
theta = pi/3
state = [cis(-theta/2)*state[0], cis(theta/2)*state[1]]

state

[(-0.4054622662672945+0.8615934406792087j),
 (0.2278412353117483+0.20333584313536407j)]

## THE Y GATE - Rotate the amplitude on the 0 side counterclockwise by 90° and the amplitude on the 1 side clockwise by 90°; Swap the two amplitudes.

In [35]:
state = [-1j*state[1], 1j*state[0]]

state

[(0.20333584313536407-0.2278412353117483j),
 (-0.8615934406792087-0.4054622662672945j)]

## Simulate applying the Hadamard gate to a single-qubit state:

In [36]:
(a, b, c, d) = (1/sqrt(2), 1/sqrt(2), 1/sqrt(2), -1/sqrt(2))
state = [a*state[0] + b*state[1], c*state[0] + d*state[1]]

state

[(-0.4654584109908178-0.44781320051572654j),
 (0.7530187180694179+0.12559703547001239j)]

## Function to create a default single-qubit state

In [38]:
def init_state():
  state = [0 for _ in range(2)]
  state[0] = 1
  return state

## Visualizing a quantum state with a state table

In [40]:
# Creates nested lists with outcome, amplitude, direction, magnitude, and probability
def to_table(s, decimals=5):
  table = [
    [k, s[k], math.atan2(s[k].imag, s[k].real) / (2 * pi) * 360, abs(s[k]),
    abs(s[k]) ** 2] for k in range(len(s))]

  # Rounds the values (the default number of digits is five)
  table_r = [[round(x, decimals) if isinstance(x, float) else round(
    x.real) + 1j * round(x.imag, decimals) if isinstance(x,complex) else
    x for x in table[k]] for k in range(len(table))]
  return table_r

# Creates a function that prints the state table for a given state
def print_state(state, decimals=5):
  print(*to_table(state, decimals),sep='\n')

In [41]:
# initialize single qubit quantum state
state = init_state()
print_state(state)

[0, 1, 0.0, 1, 1]
[1, 0, 0.0, 0, 0]


In [45]:
## encode four values of a gate
gate = [[a, b], [c, d]]
gate

[[0.7071067811865475, 0.7071067811865475],
 [0.7071067811865475, -0.7071067811865475]]

In [43]:
## Code implementations of basic single-qubit gates
x = [[0, 1], [1, 0]]
z = [[1, 0], [0, -1]]

# gate functions

def phase(theta):
  return [[1, 0], [0, complex(cos(theta), sin(theta))]]

h = [[1/sqrt(2), 1/sqrt(2)], [1/sqrt(2), -1/sqrt(2)]]

def rz(theta):
  return [[complex(cos(theta / 2), -sin(theta / 2)), 0],
  [0, complex(cos(theta / 2), sin(theta / 2))]]
  y = [[0, complex(0, -1)], [complex(0, 1), 0]]

def rx(theta):
  return [[cos(theta/2), complex(0, -sin(theta/2))],
  [complex(0, -sin(theta/2)), cos(theta/2)]]

def ry(theta):
  return [[cos(theta/2), -sin(theta/2)], [sin(theta/2), cos(theta/2)]]

In [44]:
# compute the new amplitude for outcome 0
print(gate[0][0]*state[0] + gate[0][1]*state[1])

# and now for outcome 1
print(gate[1][0]*state[0] + gate[1][1]*state[1])

0.7071067811865475
0.7071067811865475


## Simulating applying gate transformations to a single-qubit gate

In [47]:
def transform(state, gate):
  assert(len(state) == 2) # checks state has two values
  z0 = state[0]
  z1 = state[1]
  state[0] = gate[0][0]*z0 + gate[0][1]*z1 # new value of 1st amplitude
  state[1] = gate[1][0]*z0 + gate[1][1]*z1 # new value of 2nd amplitude

## Single-qubit circuits

In [49]:
s = init_state()

transform(s, ry(2*pi/3))
transform(s, x)
transform(s, phase(pi/3))
transform(s, h)

print_state(s)

[0, (1+0.30619j), 21.20602, 0.84647, 0.71651]
[1, -0.30619j, -35.10391, 0.53244, 0.28349]


## Simulating measurement of single-qubit states

In [51]:
# simulate 10 measurements by getting 10 samples from the
# probability distribution determined by the state
from random import choices
from collections import Counter

samples = choices(range(len(s)), [abs(s[k])**2 for k in range(len(s))], k=10)
print(samples)

# count samples
for (k, v) in Counter(samples).items():
  print(str(k) + ' -> ' + str(v))

[0, 0, 0, 0, 0, 0, 1, 0, 0, 0]
0 -> 9
1 -> 1


In [52]:
## perform 1_000 samples
samples = choices(range(len(s)), [abs(s[k])**2 for k in range(len(s))], k=1000)

for (k, v) in Counter(samples).items():
  print(str(k) + ' -> ' + str(v))

0 -> 730
1 -> 270


## Encoding the uniform distribution in a single-qubit quantum state

In [54]:
state = init_state()

transform(state, h) # H gate

print_state(state) # print state

# take 10 samples
samples = choices(range(len(state)), [abs(state[k])**2 for k in
range(len(state))], k=10)

for (k, v) in Counter(samples).items():
  print(str(k) + ' -> ' + str(v))

[0, 0.70711, 0.0, 0.70711, 0.5]
[1, 0.70711, 0.0, 0.70711, 0.5]
1 -> 5
0 -> 5


## Encoding a Bernoulli distribution in a single-qubit quantum state

In [55]:
# simulate encoding the Bernoulli distribution for p = 0.7
from math import acos

p = 0.7

theta = 2*acos(sqrt(p)) # theta according to p

s = init_state()

transform(s, ry(theta)) # apply Ry gate

print_state(s)

[0, 0.83666, 0.0, 0.83666, 0.7]
[1, 0.54772, 0.0, 0.54772, 0.3]


## Encoding a number with a single qubit

In [57]:
# create a single-qubit state and encodes the value x = 273.5 in
# the magnitude of the amplitude corresponding to outcome 0
x = 273.5

theta = 2*acos(x/1000) # find theta according to x value

state = init_state()

transform(state, ry(theta)) # apply Ry gate

print_state(state)

[0, 0.2735, 0.0, 0.2735, 0.0748]
[1, 0.96187, 0.0, 0.96187, 0.9252]


## Brief aside: fixing to_table()

- Missing basis label
- Real part of complex number missing

In [59]:
def to_table(s, decimals=5):
    # Build table: [index, label, amplitude, phase°, |amp|, |amp|²]
    table = [
        [k,
         str(k),                               # basis-label column
         s[k],
         math.atan2(s[k].imag, s[k].real) / (2*pi) * 360,
         abs(s[k]),
         abs(s[k])**2]
        for k in range(len(s))
    ]

    # Round everything nicely
    table_r = []
    for row in table:
        rounded_row = []
        for x in row:
            if isinstance(x, float):
                rounded_row.append(round(x, decimals))
            elif isinstance(x, complex):
                rounded_row.append(
                    complex(round(x.real, decimals), round(x.imag, decimals))
                )
            else:                               # int / str stay unchanged
                rounded_row.append(x)
        table_r.append(rounded_row)
    return table_r

## ENCODING A NUMBER IN THE ANGLE OF AN AMPLITUDE

In [ ]:
# encode the value x = 273.5 in the phase of the amplitude corresponding
# to outcome 1, we will use v = x/1000. We can use the Hadamard gate followed by a
# phase gate with the angle π v: P(π x/1000)H.

In [60]:
x = 273.5 # desired encoded value

theta = pi*x/1000 # get theta for x val

state = init_state()

transform(state, h) # Apply H gate
transform(state, phase(theta)) # apply phase gate

print_state(state)

[0, '0', 0.70711, 0.0, 0.70711, 0.5]
[1, '1', (0.46176+0.53552j), 49.23, 0.70711, 0.5]


In [64]:
# check that theta_1 holds encoded value
direction_1 = math.atan2(state[1].imag, state[1].real) # gets theta_1

round(direction_1/pi*1000, 1) # solves for x

273.5